In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import gc

In [2]:
Data = pd.read_csv("./Source/CarOR_VanAvailability_OA_AnalysisCombined.csv",index_col=['PowerLiftRatio,Perc','Output Areas'])
# CSV file taken from 'https://www.ons.gov.uk/datasets/TS045/editions/2021/versions/4/filter-outputs/21f5ef3b-68bf-44ee-b610-f654febc6e8d#get-data'

In [3]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6799680 entries, (100, 'E00000001') to (170, 'W00010697')
Data columns (total 11 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   1 car or van in household            int64  
 1   2 cars or vans in household          int64  
 2   3 or more cars or vans in household  int64  
 3   No cars or vans in household         int64  
 4   TotalHousehold                       int64  
 5   TotalCars                            int64  
 6   PowerProfile_kWe                     object 
 7   NumberOfCars                         object 
 8   TotalCarsCharged                     float64
 9   EV_SufficiencyFlag                   object 
 10  EV_ChargingPenetrationRate           float64
dtypes: float64(2), int64(6), object(3)
memory usage: 7.5 GB


In [4]:
DropColumns = ['1 car or van in household', '2 cars or vans in household','3 or more cars or vans in household', 'No cars or vans in household',
                'TotalHousehold', 'TotalCars', 'PowerProfile_kWe', 'NumberOfCars','TotalCarsCharged', 'EV_SufficiencyFlag']
Data.drop(columns=DropColumns,inplace=True)

In [5]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6799680 entries, (100, 'E00000001') to (170, 'W00010697')
Data columns (total 1 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   EV_ChargingPenetrationRate  float64
dtypes: float64(1)
memory usage: 100.2 MB


In [6]:
gc.collect()

0

In [7]:
SHAPEFILE = "./ShapeFiles/Output_Areas_2021_EW_BFC_V8_8958595138613564964/OA_2021_EW_BFC_V8.shp"
GeoDF = gpd.read_file(SHAPEFILE)[['OA21CD','geometry']]
GeoDF.columns = ['OA_21_Code','geometry']
GeoDF.head()

,OA_21_Code,geometry
0,E00000001,"POLYGON ((532303.492 181814.110, 532301.242 18..."
1,E00000003,"POLYGON ((532213.378 181846.192, 532213.954 18..."
2,E00000005,"POLYGON ((532180.131 181763.020, 532177.000 18..."
3,E00000007,"POLYGON ((532201.292 181668.180, 532205.000 18..."
4,E00000010,"POLYGON ((532127.958 182133.192, 532106.413 18..."


In [8]:
GeoDF.info(memory_usage='deep')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   OA_21_Code  188880 non-null  object  
 1   geometry    188880 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 13.3 MB


In [9]:
gc.collect()

0

In [10]:
def PlotFigure(DF,UpLift):
    Title = f'EV charging penetration OA\nPower Uplift of +{UpLift-100}%.'
    col = 'EV_ChargingPenetrationRate'
    cmap = 'RdYlGn'
    fig, ax = plt.subplots(1,figsize=(9,16),dpi=1000)
    ax.axis('off')
    DF.plot(column=col, ax=ax, edgecolor='white', linewidth=0.01, cmap=cmap)
    ax.set_title(Title, fontdict={'fontsize': '14', 'fontweight': '3'})# Add a title
    sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=0, vmax=100), cmap=cmap)# Create colorbar as a legend
    # Empty array for the data range
    sm._A = []
    cbaxes = fig.add_axes([0.18,0.35,0.02,0.2])
    cbar = fig.colorbar(sm, cax=cbaxes)
    plt.savefig(f"./Pictures/EV_Charging_Penetration_OA_{UpLift}.png")
    plt.close('all')
    gc.collect()

In [11]:
PowerLifts = set()
for item in Data.index:
    PowerLifts.add(item[0])
PowerLifts = sorted(PowerLifts)

In [13]:
gc.collect()
for ChargeUpLift in tqdm(PowerLifts[4::5]):
    SelectedData = Data.loc[ChargeUpLift].reset_index()
    MergedDF = pd.merge(left=GeoDF, right=SelectedData, how='left', left_on='OA_21_Code', right_on='Output Areas')
    MergedDF.drop(columns=['Output Areas'],inplace=True)
    PlotFigure(MergedDF,ChargeUpLift)
    gc.collect()

100%|████████████████████████████████████████████████████████████████████| 7/7 [07:53<00:00, 67.65s/it]
